<a href="https://colab.research.google.com/github/SangMin316/Bigdata_AI_idea_contest/blob/main/Creat_EVstockdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

서울시 전기차 충전소를 8개의 기준으로 나누는 코드입니다.

기준이 3개(급속인지 아닌지, 24시간인지 아닌지, 제한조건이 있는지 없는지)에 따라
8개로 구분했습니다.



In [ ]:
import pandas as pd
df = pd.read_excel('seoul_EVstock (1).xlsx')

data 살펴보기


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11071 entries, 0 to 11070
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4566 non-null   float64
 1   운영기관         11071 non-null  object 
 2   충전소          11071 non-null  object 
 3   충전기 ID       11071 non-null  int64  
 4   타입           11071 non-null  object 
 5   지역           11071 non-null  object 
 6   시군구          11071 non-null  object 
 7   주소           11071 non-null  object 
 8   이용가능시간       10267 non-null  object 
 9   제한           5849 non-null   object 
 10  종류           297 non-null    object 
 11  비고           1804 non-null   object 
 12  Unnamed: 12  7584 non-null   object 
 13  dong         11071 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 1.2+ MB


In [ ]:
df['타입'].value_counts(10)
#검색을 해봤는데, ac완속을 제외한 충전기는 다 급속이었습니다.

AC완속               0.871918
DC차데모+AC3상+DC콤보    0.063951
DC콤보               0.048686
DC차데모+DC콤보         0.011652
DC차데모+AC3상         0.002619
AC3상               0.000813
DC차데모              0.000361
Name: 타입, dtype: float64

In [ ]:
df['종류'].value_counts(dropna = False)
# 급속에도 kW수에 따라 종류가 나뉘는데 NaN값이 압도적으로 많아서 
# 충전기는 급속이냐 완속이냐로 나누겠습니다.

NaN            10774
급속(50kW)         116
급속(200kW동시)       72
급속(100kW멀티)       58
급속(100kW단독)       34
급속(400kW동시)        8
급속(100kW동시)        8
s                  1
Name: 종류, dtype: int64

In [ ]:
df['비고'].value_counts(10,dropna = False)
# 제한조건이 없는 충전기가 83퍼센트나 되네요

NaN                                  0.837052
입주민만 사용가능 거주자외 출입제한                  0.064493
입주민만 사용가능 거주자 외 출입제한                 0.034234
입주민 등 거주자 외 출입 제한                    0.010207
공동주택 입주민 전용                          0.005058
                                       ...   
1시간무료, 전기차 주차시 50% 할인                0.000090
30분 무료 이후 10분당1000원                  0.000090
최초 1시간 1500(승용차기준) 10분 250원          0.000090
주차비 5분 무료 이후 5분당 50원(친환경차 50% 감면)    0.000090
주차비 5분 50원(친환경차 50% 감면)              0.000090
Name: 비고, Length: 121, dtype: float64

In [ ]:
df['제한'].value_counts(10,dropna = False)

NaN                             0.471683
거주자외 출입제한                       0.263662
시설 상황에 따라 이용이 제한될 수 있음          0.129347
입주자외 출입제한 (긴급상황 이용가능)           0.035769
해당 시설 정책에 따라 이용이 불가할 수 있습니다     0.031343
해당 시설 정책에 따라 이용이 불가할 수 있습니다.    0.031162
사업장 거주자외 출입제한                   0.017794
학교 교직원으로 제한                     0.004155
RFID 카드만 가능                     0.003884
금융기관 시설 이용자로 제한                 0.001536
회원전용                            0.001445
아파트                             0.001174
실내(부분개방)_일부 이용자 제한 가능           0.000813
어댑터 사용 불가                       0.000723
주유소 시설 이용자로 제한                  0.000723
업무용 빌딩(공장 등) 입주자 대상 개방          0.000632
마트(쇼핑몰) 이용자, 상가 입주자로 사용 제한      0.000542
숙박 시설 이용자로 제한                   0.000542
관람장 시설 이용자로 제한                  0.000361
소방서관용차 전용                       0.000271
직원 전용, 외부인 사용불가                 0.000271
실내(부분개방)_거주자외 출입제한              0.000271
부분공용                            0.000271
교육원 시설 이용자로 제한                  0.000181
병원내복잡으로 내부차량 사용 

In [ ]:
df['이용가능시간'].value_counts( dropna = False)
# 이용가능한 시간도 기준을 다 입력하기에는 무리가 있어서 
# 상위 4개와 기존에 만들었던 기준들을 사용하겠습니다.

24시간 이용가능                         5787
09:00~18:00                       1700
24시간 이용가능,입주민만 사용가능 거주자외출입제한       998
NaN                                804
24시간                               595
                                  ... 
주중 08시~22시, 토 08시~13시, 공휴일 미개방       1
평일09:00~12:00                        1
점포영업시간                               1
06:00~24:00 이용가능                     1
09:00~20:00                          1
Name: 이용가능시간, Length: 118, dtype: int64

In [ ]:
fulltime_ratio = 5787 + 998 + 804 + 595

In [ ]:
a = len(df[df['이용가능시간'] == '~'])
b = len(df[df['이용가능시간'] == '00:00 ~ 23:59' ])
c = len(df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ])
d = len(df[df['이용가능시간'] == '주증/주말 : 24시간' ])
e = len([df['이용가능시간'] == '입주민 전용 24시간' ])
others = (a+b+c+d+e)
print((fulltime_ratio + others)/11071)

# 전에 24시간 가능한 가능한 충전소가 약 77퍼센트 존재한다고 봤는데
# 이번에는 약 76.2퍼가 나왔습니다. 약 1퍼센트의 오차는 어디서 발생했는지 잘 모르겠습니다.. 

0.7626230692801012


위 조건들을 고려해서 충전기를 분류해보겠습니다.

In [ ]:
# 급속이냐 완속이냐 나눠주는 함수입니다.
def classify_fast(df):
  f_df = df[df['타입'] != 'AC완속'] 
  idxs = f_df.index
  s_df = df.drop(idxs)
  return f_df, s_df # f_df는 급속충전기, s_df는 완속충전기

# 24시간 이용가능하냐 그렇지 않냐 나눠주는 함수 입니다.
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  f7 = df[df['이용가능시간'] == '주중/주말 : 24시간']
  f8 = df[df['이용가능시간'] == '24시간(단, 22시이후근무자운영' ]
   
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]
  c3 = df[df['이용가능시간'] == '입주민 전용 24시간' ]
  c4 = df[df['이용가능시간'] == '24시간 이용가능(외부인 미개방)' ]
 
  totalf = pd.concat([f1,f2,f3,f4,f5,f6,f7,f8])
  temporary_c = pd.concat([c1, c2, c3,c4])
  full = pd.concat([totalf, temporary_c]) #24시간 이용가능한 충전기 입니다.
  idx = full.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [ ]:
# 제한조건이 있는지 없는지 나눠주는 함수입니다.
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c3 = df[df['비고'] == '입주민 등 거주자 외 출입 제한' ]
  c4 = df[df['비고'] == '공동주택 입주민 전용' ]
  c5 = df[df['비고'] == '거주자외 출입제한' ]
  c6 = df[df['비고'] == '입주민으로 사용제한' ]
  c7 = df[df['제한'] == '거주자외 출입제한' ]
  c8 = df[df['제한'] == '입주자외 출입제한 (긴급상황 이용가능)' ]  
  c9 = df[df['제한'] == '사업장 거주자외 출입제한' ]  
  c10 = df[df['제한'] == '학교 교직원으로 제한 ' ]  
  c11 = df[df['제한'] == '회원전용' ]
  c12 = df[df['제한'] == '아파트' ]  
  
  newc = pd.concat( [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12])
  newc = newc.drop_duplicates()  # 비고랑 제한이랑 중복되는 놈들이 있을 수 있기에 빼줍니다.
  # fulltime data가 아니면 tmeporary_c데이터를 포함하고 있지 않기에 newc를 바로 출력하면됩니다.
  # 반번 fulltime데이터면 temporary_c 데이터를 포함하고 있기에 temporary_c데이터도 고려해줘야합니다.
  if len(df[df['이용가능시간'] == '24시간']) != 0: # fulltime이면 
    print('full data')
    c = pd.concat([temporary_c, newc])
    c = c.drop_duplicates() # 이용가능한시간이랑 제한이랑 겹치는 애들이 있을 수 있으니 중복되는 애들은 빼줍시다.
    nonc = df.drop(c.index)
    
  else:
    print('nonfulltime')
    nonc = df.drop(newc.index)
    c = newc
          
  return c,nonc

In [ ]:
# 앞에서 만든 분류 함수들을 순차적으로 돌려주는 함수입니다.
def classify_all(df):
  # fast slow classify
  fast, slow = classify_fast(df)
  # time classify
  fast_full , fast_nonfull , temporary_fast_cons = classify_time(fast) 
  slow_full , slow_nonfull , temporary_slow_cons = classify_time(slow)
  print('ff',len(fast_full))
  print('fnf',len(fast_nonfull))
  print('sf',len(slow_full))
  print('snf',len(slow_nonfull))
  print('total',len(fast_full) + len(fast_nonfull) +len(slow_full) + len(slow_nonfull) )
  # constraints classify
  ffc, ffnc = classify_constraints(fast_full,temporary_fast_cons)
  print('a')
  nfc, nfnc = classify_constraints(fast_nonfull,temporary_fast_cons)
  print('d')
  fsc, fsnc = classify_constraints(slow_full, temporary_slow_cons)
  print('c')
  nsc, nsnc = classify_constraints(slow_nonfull, temporary_slow_cons)
  print('d')
  return ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc

In [ ]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df)

ff 931
fnf 487
sf 6802
snf 2851
total 11071
full data
a
nonfulltime
d
full data
c
nonfulltime
d


In [ ]:
ffc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 203 to 5955
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   11 non-null     float64
 1   운영기관         38 non-null     object 
 2   충전소          38 non-null     object 
 3   충전기 ID       38 non-null     int64  
 4   타입           38 non-null     object 
 5   지역           38 non-null     object 
 6   시군구          38 non-null     object 
 7   주소           38 non-null     object 
 8   이용가능시간       38 non-null     object 
 9   제한           38 non-null     object 
 10  종류           0 non-null      object 
 11  비고           0 non-null      object 
 12  Unnamed: 12  15 non-null     object 
 13  dong         38 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory usage: 4.5+ KB


In [ ]:
ffnc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 5001
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   378 non-null    float64
 1   운영기관         893 non-null    object 
 2   충전소          893 non-null    object 
 3   충전기 ID       893 non-null    int64  
 4   타입           893 non-null    object 
 5   지역           893 non-null    object 
 6   시군구          893 non-null    object 
 7   주소           893 non-null    object 
 8   이용가능시간       893 non-null    object 
 9   제한           220 non-null    object 
 10  종류           224 non-null    object 
 11  비고           220 non-null    object 
 12  Unnamed: 12  692 non-null    object 
 13  dong         893 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 104.6+ KB


In [ ]:
a =len(ffc) + len(ffnc) + len(nfc) +  len(nfnc) + len(fsc) +  len(fsnc) +  len(nsc) + len(nsnc)
print(a)
# 중복과 누락없이 잘 저장되었습니다.

11071


In [ ]:
# 중복되는 친구들을 고려하지 못해서 해매다가 겨우 찾았습니다.. 

In [ ]:
# dong을 추출합니다.
a = df['dong'].values
index = set(a)
print(a)

['성수1가1동' '성수1가1동' '마장동' ... '창1동' '창1동' '창4동']


In [ ]:
EVstockdf = pd.DataFrame(index = index)

EVstockdf['ffnc'] =  ffnc['dong'].value_counts()
EVstockdf['ffc'] =  ffc['dong'].value_counts()
EVstockdf['nfnc'] =  nfnc['dong'].value_counts()
EVstockdf['nfc'] =  nfc['dong'].value_counts()
EVstockdf['fsnc'] =  fsnc['dong'].value_counts()
EVstockdf['fsc'] =  fsc['dong'].value_counts()
EVstockdf['nsnc'] =  nsnc['dong'].value_counts()
EVstockdf['nsc'] =  nsc['dong'].value_counts()

In [ ]:
EVstockdf.head()

,ffnc,ffc,nfnc,nfc,fsnc,fsc,nsnc,nsc
신대방1동,1.0,NaN,1.0,NaN,5.0,13.0,5.0,13.0
시흥5동,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
청담동,1.0,NaN,NaN,2.0,13.0,19.0,26.0,NaN
신정4동,NaN,NaN,2.0,NaN,9.0,2.0,NaN,NaN
항동,NaN,NaN,NaN,NaN,NaN,11.0,9.0,10.0


In [ ]:
EVstockdf.info()
# 행정동은 426개인데, 저희가 정리한 데이터는 418개네여... 어디가 잘못되었는지 ㅠ
# 짐작가는 곳은 'hangjungdong'데이터에서 생활인구데이터와 겹치치 않은 곳을 추출했는데
# 법정동이름이랑 행정동이름이랑 같은 곳이 존재했던 것 같습니다..
# 일단 2개 차이니깐 생활인구데이터를 합치면서 진행해보겠습니다.

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 신대방1동 to 미성동
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ffnc    276 non-null    float64
 1   ffc     20 non-null     float64
 2   nfnc    147 non-null    float64
 3   nfc     50 non-null     float64
 4   fsnc    327 non-null    float64
 5   fsc     285 non-null    float64
 6   nsnc    300 non-null    float64
 7   nsc     196 non-null    float64
dtypes: float64(8)
memory usage: 29.4+ KB


In [ ]:
EVstockdf.to_excel('EVstock1.xlsx')